# Neo4j tasks:

#### Importing libraries

In [1]:
from neo4j import GraphDatabase

#### Creating a connection to database:

In [2]:
uri = "neo4j+ssc://b54de357.databases.neo4j.io"
username = "neo4j"
password = "kwngn-c0-P0HUB_10uApOvo7fP87YknHCuhzG8b0gCM"

In [3]:
driver = GraphDatabase.driver(uri,auth=(username,password))

In [4]:
with driver.session() as session:
    pass;

## 1. Graph Creation:

### Build a graph database to represent a social network, with nodes for users and relationships for friendships. Create nodes for users and relationships to model their interactions.

In [5]:
def create_person_node(transaction,name,age):
    transaction.run("CREATE (:Person {name:$name, age:$age})", name=name, age=age)
    
with driver.session() as session:
    session.execute_write(create_person_node,"Martin",21)
    session.execute_write(create_person_node,"Alice",20)
    session.execute_write(create_person_node,"Jane",22)
    session.execute_write(create_person_node,"Victor",19)
    session.execute_write(create_person_node,"Bill",24)
    session.execute_write(create_person_node,"Bob",23)

In [6]:
def create_friendship(transaction,name1,name2):
    transaction.run("""
        MATCH (person1:Person {name:$name1}),(person2:Person {name:$name2}) 
        CREATE (person1)-[:FRIEND_OF]->(person2) 
    """, name1=name1,name2=name2)

In [7]:
with driver.session() as session:
    session.execute_write(create_friendship,"Martin","Bob")
    session.execute_write(create_friendship,"Bob","Alice")
    session.execute_write(create_friendship,"Jane","Bob")
    session.execute_write(create_friendship,"Alice","Jane")
    session.execute_write(create_friendship,"Victor","Bill")
    session.execute_write(create_friendship,"Bill","Bob")
    session.execute_write(create_friendship,"Martin","Victor")
    session.execute_write(create_friendship,"Jane","Victor")
    session.execute_write(create_friendship,"Martin","Alice")

# 2. Querying Graph Data:

### Write a Cypher query to find all friends of a given user up to a certain depth (degree of separation).

In [8]:
def get_person_friends(transaction,name):
    results = transaction.run("""
        MATCH (person:Person {name:$name})-[:FRIEND_OF]->(friend:Person) 
        RETURN person.name,person.age,friend.name,friend.age
        """,name=name)
    
    return [{
            "name":result["person.name"],
             "age":result["person.age"], 
             "friend name":result["friend.name"], 
             "friend age":result["friend.age"]
            } 
        for result in results]

In [9]:
with driver.session() as session:
    print(session.execute_read(get_person_friends,"Martin"))

[{'name': 'Martin', 'age': 21, 'friend name': 'Alice', 'friend age': 20}, {'name': 'Martin', 'age': 21, 'friend name': 'Victor', 'friend age': 19}, {'name': 'Martin', 'age': 21, 'friend name': 'Bob', 'friend age': 23}]


In [10]:
def get_person_friends_of_friends(transaction,name):
    results = transaction.run("""
        MATCH (person:Person {name:$name})-[:FRIEND_OF*2]->(fof) 
        WHERE fof.name <> $name
        RETURN person.name,person.age,fof.name,fof.age
        """,name=name)
    
    return [{
            "name":result["person.name"],
             "age":result["person.age"], 
             "friend name":result["fof.name"], 
             "friend age":result["fof.age"]
            } 
        for result in results]

In [11]:
with driver.session() as session:
    print(session.execute_read(get_person_friends_of_friends,"Martin"))

[{'name': 'Martin', 'age': 21, 'friend name': 'Jane', 'friend age': 22}, {'name': 'Martin', 'age': 21, 'friend name': 'Bill', 'friend age': 24}, {'name': 'Martin', 'age': 21, 'friend name': 'Alice', 'friend age': 20}]


### Retrieve common friends between two users in the graph.

In [12]:
def common_friends(transaction,name1,name2):
    results = transaction.run("""
        MATCH (N:Person {name:$name1}),(M:Person {name:$name2})
        WITH [ (N)--(B)--(M) WHERE B:Person|B.name] AS names
        UNWIND names AS name_of_mutual_friend
        RETURN name_of_mutual_friend
        """,name1=name1,name2=name2)
    
    return [result["name_of_mutual_friend"] for result in results]
    

In [13]:
with driver.session() as session:
    print(session.execute_read(common_friends,"Martin","Bill"))

['Bob', 'Victor']


## 3. Recommendation System:

### Implement a basic recommendation system by finding users who have similar interests based on their interactions.

In [14]:
def create_interest_node(transaction,name):
    transaction.run("CREATE (:Interest {name:$name})", name=name)
    
with driver.session() as session:
    session.execute_write(create_interest_node,"Skiing")
    session.execute_write(create_interest_node,"Diving")
    session.execute_write(create_interest_node,"Basketball")
    session.execute_write(create_interest_node,"Movies")
    session.execute_write(create_interest_node,"Theater")
    session.execute_write(create_interest_node,"Jogging")

In [15]:
def create_interest(transaction,person_name,interest):
    transaction.run("""
        MATCH (person:Person {name: $person_name}), (interest:Interest {name: $interest})
        MERGE (person)-[:INTERESTED_IN]->(interest)
    """, person_name=person_name,interest=interest)

In [16]:
with driver.session() as session:
    session.execute_write(create_interest,"Martin","Skiing")
    session.execute_write(create_interest,"Bob","Theater")
    session.execute_write(create_interest,"Jane","Skiing")
    session.execute_write(create_interest,"Alice","Basketball")
    session.execute_write(create_interest,"Victor","Theater")
    session.execute_write(create_interest,"Bill","Basketball")
    session.execute_write(create_interest,"Martin","Diving")
    session.execute_write(create_interest,"Jane","Jogging")
    session.execute_write(create_interest,"Martin","Movies")
    session.execute_write(create_interest,"Victor","Jogging")
    session.execute_write(create_interest,"Bill","Skiing")
    session.execute_write(create_interest,"Martin","Jogging")
    session.execute_write(create_interest,"Jane","Movies")
    session.execute_write(create_interest,"Bob","Basketball")

In [17]:
def persons_with_common_interests(transaction,person_name):
    results = transaction.run("""
            MATCH (person:Person {name: $person_name})-[:INTERESTED_IN]->(interest)<-[:INTERESTED_IN]-(otherPerson:Person)
            WHERE person <> otherPerson
            RETURN otherPerson.name AS possible_friend, COLLECT(interest.name) AS common_interest      
        """,person_name=person_name)
    
    return [{"possible_friend": result["possible_friend"], "common_interest": result["common_interest"]} for result in results]

In [18]:
with driver.session() as session:
    print(session.execute_read(persons_with_common_interests,"Bill"))

[{'possible_friend': 'Jane', 'common_interest': ['Skiing']}, {'possible_friend': 'Martin', 'common_interest': ['Skiing']}, {'possible_friend': 'Bob', 'common_interest': ['Basketball']}, {'possible_friend': 'Alice', 'common_interest': ['Basketball']}]


### Propose movies, books, or other items that a user might be interested in based on their connections' preferences

In [19]:
def recommended_interests(transaction,person_name):
    results = transaction.run("""
            MATCH (person:Person {name: $person_name})-[:FRIEND_OF]->(otherPerson:Person)-[:INTERESTED_IN]->(interest)
            RETURN DISTINCT interest.name
    """,person_name=person_name)
    
    return [result["interest.name"] for result in results]

In [20]:
with driver.session() as session:
    print(session.execute_read(recommended_interests,"Alice"))

['Movies', 'Jogging', 'Skiing']


## Pathfinding:

### Use graph algorithms in Neo4j to find the shortest path between two nodes in the graph.

In [21]:
def find_shortest_path(transaction,name1,name2):
    results = transaction.run("""
        MATCH (person1:Person {name:$name1}),(person2:Person {name:$name2}), path= SHORTESTPATH ((person1)-[*]->(person2)) 
        RETURN nodes(path),relationships(path)
    """,name1=name1,name2=name2)
    
    return [{"nodes":result["nodes(path)"],"relationships":result["relationships(path)"]} for result in results]

In [22]:
with driver.session() as session:
    results = session.execute_read(find_shortest_path,"Alice","Bill")
    nodes = results[0]["nodes"]
    relationships = results[0]["relationships"]

In [23]:
for node in nodes:
    print(node.get("name"))

Alice
Jane
Victor
Bill


In [24]:
for relationship in relationships:
    nodes = relationship.nodes
    print(nodes[0].get("name"),"->",nodes[1].get("name"))

Alice -> Jane
Jane -> Victor
Victor -> Bill


### Find the path of mutual friends between two users.

In [25]:
def find_mutual_friends_path(transaction,name1,name2):
    results = transaction.run("""
        MATCH (person1:Person {name: $name1})-[:FRIEND_OF]-(mutualFriend:Person)-[:FRIEND_OF]-(person2:Person {name: $name2})
        RETURN COLLECT(DISTINCT mutualFriend.name) AS mutual_friends
    """,name1=name1,name2=name2)
    path = results.single()
    
    return path["mutual_friends"]

In [26]:
with driver.session() as session:
    print(session.execute_read(find_mutual_friends_path,"Alice","Victor"))

['Jane', 'Martin']
